In [2]:
import pandas as pd
import itertools
import numpy as np

In [3]:
csv= '/home/mei/nas/docker/thesis/data/csv/'

In [4]:
def round_up(x, base=5):
    return base * round(x/base)

In [5]:
print('==> Loading data from timeseries files...')
timeseries_lab = pd.read_csv(csv + 'timeserieslab.csv', low_memory=False)
timeseries_periodic = pd.read_csv(csv+ 'timeseriesperiodic.csv')

==> Loading data from timeseries files...


In [6]:
print("there are {} patients in the  and {} records in lab table.".format(len(list(timeseries_lab ['patientunitstayid'].unique())),len(timeseries_lab)))
print("there are {} patients in the  and {} records in vital periodic table.".format(len(list(timeseries_periodic ['patientunitstayid'].unique())),len(timeseries_periodic)))


there are 9928 patients in the  and 753247 records in lab table.
there are 9928 patients in the  and 2743498 records in vital periodic table.


In [8]:
timeseries_lab = timeseries_lab[timeseries_lab['labresultoffset'] >= 0]
timeseries_periodic = timeseries_periodic[timeseries_periodic['observationoffset'] >= 0]


In [9]:
def reconfigure_timeseries(timeseries, offset_column, feature_column=None, test=False):
    """
    Reconfigure timeseries data by setting multi-index and pivoting if necessary.
    """
    if test:
        timeseries = timeseries.iloc[:5000]  # Limit for testing
    timeseries.set_index(['patientunitstayid', pd.to_timedelta(timeseries[offset_column], unit='min')], inplace=True)
    timeseries.drop(columns=offset_column, inplace=True)
    if feature_column:
        timeseries = timeseries.pivot_table(columns=feature_column, index=timeseries.index)
    timeseries.index = pd.MultiIndex.from_tuples(timeseries.index, names=['patient', 'time'])
    return timeseries

In [10]:
test=False
print('==> Reconfiguring lab timeseries...')
timeseries_lab = reconfigure_timeseries(timeseries_lab, 'labresultoffset', 'labname', test)
timeseries_lab.columns = timeseries_lab.columns.droplevel()

print('==> Reconfiguring periodic timeseries...')
timeseries_periodic = reconfigure_timeseries(timeseries_periodic, 'observationoffset', test=test)


==> Reconfiguring lab timeseries...
==> Reconfiguring periodic timeseries...


In [11]:

print('==> Combining data together...')
merged = pd.concat([timeseries_lab, timeseries_periodic], axis=0, sort=False)

==> Combining data together...


In [12]:
merged

-basos  -eos  -lymphs  -monos  -polys  ALT (SGPT)  \
patient time                                                                 
252784  0 days 00:20:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 00:39:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 01:20:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 01:38:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 02:33:00     NaN   NaN      NaN     NaN     NaN         NaN   
...                         ...   ...      ...     ...     ...         ...   
3347496 0 days 08:32:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:27:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:22:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:52:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:57:00     NaN   NaN      NaN     NaN     NaN         NaN   

                         AST (SGOT)  BUN  Base Excess  FiO2  ...  sao2  \
patient time                                                 ...         
252784  0 days 00:20:00         NaN  NaN        -25.6  28.0  ...   NaN   
        0 days 00:39:00         NaN  NaN          NaN   NaN  ...   NaN   
        0 days 01:20:00         NaN  NaN          NaN   NaN  ...   NaN   
        0 days 01:38:00         NaN  NaN          NaN   NaN  ...   NaN   
        0 days 02:33:00         NaN  NaN          NaN   NaN  ...   NaN   
...                             ...  ...          ...   ...  ...   ...   
3347496 0 days 08:32:00         NaN  NaN          NaN   NaN  ...  98.0   
        0 days 08:27:00         NaN  NaN          NaN   NaN  ...  98.0   
        0 days 08:22:00         NaN  NaN          NaN   NaN  ...  98.0   
        0 days 08:52:00         NaN  NaN          NaN   NaN  ...  98.0   
        0 days 23:57:00         NaN  NaN          NaN   NaN  ...   NaN   

                         heartrate  respiration  cvp  systemicsystolic  \
patient time                                                             
252784  0 days 00:20:00        NaN          NaN  NaN               NaN   
        0 days 00:39:00        NaN          NaN  NaN               NaN   
        0 days 01:20:00        NaN          NaN  NaN               NaN   
        0 days 01:38:00        NaN          NaN  NaN               NaN   
        0 days 02:33:00        NaN          NaN  NaN               NaN   
...                            ...          ...  ...               ...   
3347496 0 days 08:32:00       89.0          NaN  NaN               NaN   
        0 days 08:27:00       94.0          NaN  NaN               NaN   
        0 days 08:22:00       95.0          NaN  NaN               NaN   
        0 days 08:52:00       91.0          NaN  NaN               NaN   
        0 days 23:57:00       78.0         23.0  NaN               NaN   

                         systemicdiastolic  systemicmean  st1  st2  st3  
patient time                                                             
252784  0 days 00:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 00:39:00                NaN           NaN  NaN  NaN  NaN  
        0 days 01:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 01:38:00                NaN           NaN  NaN  NaN  NaN  
        0 days 02:33:00                NaN           NaN  NaN  NaN  NaN  
...                                    ...           ...  ...  ...  ...  
3347496 0 days 08:32:00                NaN           NaN  NaN  NaN  NaN  
        0 days 08:27:00                NaN           NaN  NaN  NaN  NaN  
        0 days 08:22:00                NaN           NaN  NaN  NaN  NaN  
        0 days 08:52:00                NaN           NaN  NaN  NaN  NaN  
        0 days 23:57:00                NaN           NaN  NaN  NaN  NaN  

[2817023 rows x 57 columns]

In [13]:
possible_value_ranges = {
   "temperature": (32,43),        # Temperature
    "sao2": (40, 100),             # SpO2
    "heartrate": (30, 400),        # Heart rate
    "respiration": (0, 60),        # Resp. rate
    "cvp": (0, 20),                # CVP
    
    # "etco2": (0, 60),             # EtCO2
    "systemicsystolic": (40, 300), # BP systolic
    "systemicdiastolic": (20, 150),# BP diastolic
    "systemicmean": (30, 200),     # BP mean
    
}

In [14]:
def filter_vital_signs(data, ranges):
    for column, (min_val, max_val) in ranges.items():
        if column in data.columns:
            data = data[(data[column].isna()) | ((data[column] >= min_val) & (data[column] <= max_val))]
    return data

merged = filter_vital_signs(merged, possible_value_ranges)


In [15]:
# 计算 0.1% 和 99.9% 分位数
low_quantile = merged.quantile(0.001, numeric_only=True)  # 0.1% 分位数
high_quantile= merged.quantile(0.999, numeric_only=True)  # 99.9% 分位数

# 仅保留在 [0.1%, 99.9%] 之间的值
merged = merged[(merged >= low_quantile) & (merged <= high_quantile)]

print("select valid vlaue of vital signs")
print("There are {} patients and {} records in the vital periodic table.".format(
    merged.index.get_level_values('patient').nunique(),
    len(merged)
))

select valid vlaue of vital signs
There are 9928 patients and 2760127 records in the vital periodic table.


In [16]:
merged

-basos  -eos  -lymphs  -monos  -polys  ALT (SGPT)  \
patient time                                                                 
252784  0 days 00:20:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 00:39:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 01:20:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 01:38:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 02:33:00     NaN   NaN      NaN     NaN     NaN         NaN   
...                         ...   ...      ...     ...     ...         ...   
3347496 0 days 08:32:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:27:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:22:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:52:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:57:00     NaN   NaN      NaN     NaN     NaN         NaN   

                         AST (SGOT)  BUN  Base Excess  FiO2  ...  sao2  \
patient time                                                 ...         
252784  0 days 00:20:00         NaN  NaN        -25.6  28.0  ...   NaN   
        0 days 00:39:00         NaN  NaN          NaN   NaN  ...   NaN   
        0 days 01:20:00         NaN  NaN          NaN   NaN  ...   NaN   
        0 days 01:38:00         NaN  NaN          NaN   NaN  ...   NaN   
        0 days 02:33:00         NaN  NaN          NaN   NaN  ...   NaN   
...                             ...  ...          ...   ...  ...   ...   
3347496 0 days 08:32:00         NaN  NaN          NaN   NaN  ...  98.0   
        0 days 08:27:00         NaN  NaN          NaN   NaN  ...  98.0   
        0 days 08:22:00         NaN  NaN          NaN   NaN  ...  98.0   
        0 days 08:52:00         NaN  NaN          NaN   NaN  ...  98.0   
        0 days 23:57:00         NaN  NaN          NaN   NaN  ...   NaN   

                         heartrate  respiration  cvp  systemicsystolic  \
patient time                                                             
252784  0 days 00:20:00        NaN          NaN  NaN               NaN   
        0 days 00:39:00        NaN          NaN  NaN               NaN   
        0 days 01:20:00        NaN          NaN  NaN               NaN   
        0 days 01:38:00        NaN          NaN  NaN               NaN   
        0 days 02:33:00        NaN          NaN  NaN               NaN   
...                            ...          ...  ...               ...   
3347496 0 days 08:32:00       89.0          NaN  NaN               NaN   
        0 days 08:27:00       94.0          NaN  NaN               NaN   
        0 days 08:22:00       95.0          NaN  NaN               NaN   
        0 days 08:52:00       91.0          NaN  NaN               NaN   
        0 days 23:57:00       78.0         23.0  NaN               NaN   

                         systemicdiastolic  systemicmean  st1  st2  st3  
patient time                                                             
252784  0 days 00:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 00:39:00                NaN           NaN  NaN  NaN  NaN  
        0 days 01:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 01:38:00                NaN           NaN  NaN  NaN  NaN  
        0 days 02:33:00                NaN           NaN  NaN  NaN  NaN  
...                                    ...           ...  ...  ...  ...  
3347496 0 days 08:32:00                NaN           NaN  NaN  NaN  NaN  
        0 days 08:27:00                NaN           NaN  NaN  NaN  NaN  
        0 days 08:22:00                NaN           NaN  NaN  NaN  NaN  
        0 days 08:52:00                NaN           NaN  NaN  NaN  NaN  
        0 days 23:57:00                NaN           NaN  NaN  NaN  NaN  

[2760127 rows x 57 columns]

In [17]:
def gen_patient_chunk(patients, merged, size=500): # 500 is a good size for LSTM
    """
    Generate patient data chunks for processing.
    """
    it = iter(patients)
    chunk = list(itertools.islice(it, size))
    while chunk:
        yield merged.loc[chunk]
        chunk = list(itertools.islice(it, size))

In [18]:
test=False


def resample_and_mask(timeseries, csv, header, mask_decay=True, decay_rate=4/3, test=False, verbose=False):
    """
    Resample timeseries data to hourly intervals and calculate masking features.
    """
    if verbose:
       print('Resampling to 5-minute intervals...')
    timeseries = timeseries.groupby(level=[0, 1]).mean()
    unstacked = timeseries.unstack(level=0)
    unstacked.index = unstacked.index.ceil(freq='5min') # 向上取整到最近 5 分钟
    resampled = unstacked.resample('5min', closed='right', label='right').mean()

    if mask_decay:
        if verbose:
            print('Calculating mask decay features...')
        mask_bool = resampled.notnull()
        mask = mask_bool.astype(int)
        mask.replace({0: np.nan}, inplace=True)    
        inv_mask_bool = ~mask_bool
        count_non_measurements = inv_mask_bool.cumsum() - inv_mask_bool.cumsum().where(mask_bool).ffill().fillna(0)
        mask = mask.ffill().fillna(0) / (count_non_measurements * decay_rate).replace(0, 1)
    else:
        if verbose:
            print('Calculating binary mask features...')
        mask = resampled.notnull().astype(int)

    if verbose:
        print('Filling missing data forwards...')
    resampled = resampled.ffill()  # 前向填充
    resampled.index = list(range(1, len(resampled) + 1))
    mask.index = list(range(1, len(mask) + 1))

    if verbose:
        print('Filling in remaining values with zeros...')
    resampled.fillna(0, inplace=True)

    if verbose:
        print('Reconfiguring and combining features with mask features...')
        
    resampled = resampled.stack(level=1,future_stack=True).swaplevel(0, 1).sort_index(level=0)
    mask = mask.stack(level=1,future_stack=True).swaplevel(0, 1).sort_index(level=0)
    
    mask.columns = [str(col) + '_mask' for col in mask.columns]
    final = pd.concat([resampled, mask], axis=1)

    if verbose:     # if verbose=True, print the progress
        print('Saving progress...')
    if test==False:  # if test=False, save the progress
        final.to_csv(csv + 'preprocessed_timeseries.csv', mode='a', header=header)
    return final

In [19]:
import gc

patients = merged.index.unique(level=0)
gen_chunks = gen_patient_chunk(patients, merged)
header = True
print('==> Initiating main processing loop...')
for i, patient_chunk in enumerate(gen_chunks, start=1):
    final=resample_and_mask(patient_chunk,csv,header ,mask_decay=True, decay_rate=4/3, test=test, verbose=False)
    print(f'==> Processed {i * 500} patients...')
    header = False
    
    # Clear memory
    del patient_chunk
    gc.collect()

==> Initiating main processing loop...
==> Processed 500 patients...
==> Processed 1000 patients...
==> Processed 1500 patients...
==> Processed 2000 patients...
==> Processed 2500 patients...
==> Processed 3000 patients...
==> Processed 3500 patients...
==> Processed 4000 patients...
==> Processed 4500 patients...
==> Processed 5000 patients...
==> Processed 5500 patients...
==> Processed 6000 patients...
==> Processed 6500 patients...
==> Processed 7000 patients...
==> Processed 7500 patients...
==> Processed 8000 patients...
==> Processed 8500 patients...
==> Processed 9000 patients...
==> Processed 9500 patients...
==> Processed 10000 patients...


In [21]:
processed_ts = pd.read_csv(csv + 'preprocessed_timeseries.csv', index_col=[0, 1])

/tmp/ipykernel_620620/3507110383.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  processed_ts = pd.read_csv(csv + 'preprocessed_timeseries.csv', index_col=[0, 1])


In [22]:
processed_ts

-basos -eos -lymphs -monos -polys ALT (SGPT) AST (SGOT)   BUN  \
patient                                                                     
252784  1      0.8  0.4    16.6    4.4    0.0       34.0       24.0  22.0   
        2      0.8  0.4    16.6    4.4    0.0       34.0       24.0  22.0   
        3      0.8  0.4    16.6    4.4    0.0       34.0       24.0  22.0   
        4      0.8  0.4    16.6    4.4    0.0       34.0       24.0  22.0   
        5      0.8  0.4    16.6    4.4    0.0       34.0       24.0  23.0   
...            ...  ...     ...    ...    ...        ...        ...   ...   
3347496 286    1.0  1.0    15.0    4.0    0.0       13.0       19.0  42.0   
        287    1.0  1.0    15.0    4.0    0.0       13.0       19.0  42.0   
        288    1.0  1.0    15.0    4.0    0.0       13.0       19.0  42.0   
        289    1.0  1.0    15.0    4.0    0.0       13.0       19.0  42.0   
        290    1.0  1.0    15.0    4.0    0.0       13.0       19.0  42.0   

            Base Excess  FiO2  ... sao2_mask heartrate_mask respiration_mask  \
patient                        ...                                             
252784  1         -25.6  28.0  ...       1.0            1.0              1.0   
        2         -25.6  28.0  ...       1.0            1.0              1.0   
        3         -25.6  28.0  ...       1.0            1.0              1.0   
        4         -25.6  28.0  ...       1.0            1.0              1.0   
        5         -25.6  28.0  ...       1.0            1.0              1.0   
...                 ...   ...  ...       ...            ...              ...   
3347496 286         0.0  40.0  ...  0.068182            1.0              1.0   
        287         0.0  40.0  ...    0.0625            1.0              1.0   
        288         0.0  40.0  ...  0.057692            1.0              1.0   
        289         0.0  40.0  ...  0.053571            1.0              1.0   
        290         0.0  40.0  ...      0.05           0.75             0.75   

            cvp_mask systemicsystolic_mask systemicdiastolic_mask  \
patient                                                             
252784  1        0.0                   0.0                    0.0   
        2        0.0                   0.0                    0.0   
        3        0.0                   0.0                    0.0   
        4        0.0                   0.0                    0.0   
        5        0.0                   0.0                    0.0   
...              ...                   ...                    ...   
3347496 286      0.0                   0.0                    0.0   
        287      0.0                   0.0                    0.0   
        288      0.0                   0.0                    0.0   
        289      0.0                   0.0                    0.0   
        290      0.0                   0.0                    0.0   

            systemicmean_mask st1_mask st2_mask st3_mask  
patient                                                   
252784  1                 0.0      1.0      1.0      1.0  
        2                 0.0      1.0      1.0      1.0  
        3                 0.0      1.0      1.0      1.0  
        4                 0.0      1.0      1.0      1.0  
        5                 0.0      1.0      1.0      1.0  
...                       ...      ...      ...      ...  
3347496 286               0.0      0.0      0.0      0.0  
        287               0.0      0.0      0.0      0.0  
        288               0.0      0.0      0.0      0.0  
        289               0.0      0.0      0.0      0.0  
        290               0.0      0.0      0.0      0.0  

[6248419 rows x 114 columns]